In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.linear_model import LogisticRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Data preparation
df=pd.read_csv("/kaggle/input/telecom-churn/telecom_churn_data.csv")
df[['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8','last_date_of_month_9',
    'date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8','date_of_last_rech_9','date_of_last_rech_data_6',
    'date_of_last_rech_data_7','date_of_last_rech_data_8','date_of_last_rech_data_9']]=df[['last_date_of_month_6',
    'last_date_of_month_7','last_date_of_month_8','last_date_of_month_9','date_of_last_rech_6','date_of_last_rech_7',
    'date_of_last_rech_8','date_of_last_rech_9','date_of_last_rech_data_6','date_of_last_rech_data_7',
                                                'date_of_last_rech_data_8','date_of_last_rech_data_9']].apply(pd.to_datetime)
df.head()

In [ ]:
df.shape

In [ ]:
#checking missing values
df.isna().sum()/len(df)*100

In [ ]:
#removing columns which has more 70% missing data
a=df.isna().sum()/len(df)
missing_features = a[a > 0.70].index
df.drop(missing_features, axis=1, inplace=True)
df.shape

In [ ]:
col_list = df.loc[:,df.apply(pd.Series.nunique) == 1]
df.drop(col_list,axis=1,inplace=True)
df.shape

In [ ]:
df['aon_month']=round(df['aon']/30,2)
df=df.drop(['aon'],axis=1)

In [ ]:
#dropping useless column
n=[]
for i in df.columns:
    if df[i].dtypes=='datetime64[ns]':
        n.append(i)
df.drop(n,axis=1,inplace=True)

In [ ]:
# Filter high-value customers
rech_amt_6_7=(df['total_rech_num_6']*df['total_rech_amt_6']+df['total_rech_num_7']*df['total_rech_amt_7'])/2
perc=np.percentile(rech_amt_6_7,70)
high_value=df[rech_amt_6_7>=perc]
high_value.shape

In [ ]:
# Tag churners and remove attributes of the churn phase
high_value[['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9']].head()

In [ ]:
high_value['churn']=np.where(high_value[['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9']].sum(axis=1)==0,1,0)
high_value.head()

In [ ]:
high_value.churn.value_counts()/len(high_value)

In [ ]:
drop=[i for i in high_value.columns if '_9' in i]
high_value=high_value.drop(drop,axis=1)
high_value.shape

In [ ]:
high_value.isna().sum()

In [ ]:
for i in high_value.columns:
    if high_value[i].isna().sum()!=0:
        high_value[i]=high_value[i].fillna(0)

In [ ]:
high_value.isna().sum()

In [ ]:
# Before PCA standarization is important
X=high_value.drop(['churn'],axis=1)
y=high_value['churn']
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

In [ ]:
sm=SMOTE()
X_sm,y_sm=sm.fit_resample(X_train,y_train)

In [ ]:
counter=Counter(y_train)
print("Before",counter)
counter1=Counter(y_sm)
print("After",counter1)

In [ ]:
# Model Building with PCA
pca=PCA(.90)
X_tr_pca=pca.fit_transform(X_sm)
X_tr_pca.shape

In [ ]:
X_test_pca=pca.transform(X_test)
X_test_pca.shape

In [ ]:
log=LogisticRegression(solver = 'lbfgs')
log.fit(X_tr_pca,y_sm)

In [ ]:
y_pred=log.predict(X_test_pca)

In [ ]:
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.head()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Printing confusion matrix
print(confusion_matrix(y_test,y_pred))

In [ ]:
print("Logistic Regression accuracy with PCA: ",accuracy_score(y_test,y_pred))

In [ ]:
print(classification_report(y_test, y_pred))